
# Exporting Items from a Specific Zotero Group Library to a JSONL file

This Jupyter Notebook demonstrates the process of extracting data from a Zotero SQLite database and converting it into a JSONL format for further analysis and sharing. The code provided is tailored for extracting specific fields but can be modified to suit different data extraction needs.

## How to Use This Notebook
1. **Database Connection**: Modify the database path to point to your Zotero SQLite database.
2. **Queries**: The SQL queries are set up to extract specific data. Modify these queries based on your data extraction requirements.
3. **Execution**: Run the cells sequentially to extract data, transform it, and save it in the desired format.
4. **Customization**: Feel free to add or modify code blocks to suit your specific data processing and analysis needs.

**Note**: Before running, ensure that you have the necessary libraries installed (`sqlite3`, `pandas`, etc.) and that your Zotero database file is accessible from this notebook.


### Database Connection
This cell establishes a connection to the Zotero SQLite database. Make sure to update the `zotero_db_path` with the correct path to your database file.

### Data Extraction
In this cell, we execute SQL queries to extract data from the database. The extracted data includes authors, attachment paths, and other relevant fields.

### Data Transformation
Here we transform the extracted data, pivoting some of it to make it more suitable for analysis and merging different dataframes into a single dataframe.

### Saving Data
This final cell saves the transformed data into a JSONL format. You can modify the file path and name as per your requirement.

In [4]:
import os

# Define the path to the working directory in WSL format
working_dir = '/mnt/e/Google Drive/RuBase (1)/Temporality'

# Set the working directory
os.chdir(working_dir)

# Verify that the working directory is set correctly
print(f"Current working directory: {os.getcwd()}")

Current working directory: /mnt/e/Google Drive/RuBase (1)/Temporality


In [18]:
import sqlite3
import pandas as pd
from tqdm import tqdm

# Path to the Zotero SQLite database
zotero_db_path = '/mnt/e/Zotero/zotero.sqlite'

# Connect to the SQLite database
conn = sqlite3.connect(zotero_db_path)

# LibraryID to search for
library_id = 200

# Query 1: Fetch authors for items with libraryID 200
query_authors = '''
    SELECT ic.itemID, i.libraryID, c.lastName as author_last_name, c.firstName as author_first_name
    FROM itemCreators ic
    JOIN creators c ON ic.creatorID = c.creatorID
    JOIN items i ON ic.itemID = i.itemID
    WHERE i.libraryID = ?;
'''
df_authors = pd.read_sql_query(query_authors, conn, params=(library_id,))

# Query 2: Fetch attachment paths for these items
if not df_authors.empty:
    item_ids_str = ', '.join(df_authors['itemID'].astype(str).tolist())
    query_paths = f'''
        SELECT parentItemID, path 
        FROM itemAttachments 
        WHERE parentItemID IN ({item_ids_str})
    '''
    df_paths = pd.read_sql_query(query_paths, conn)
else:
    df_paths = pd.DataFrame(columns=['parentItemID', 'path'])

# Query 3: Fetch other fields for items with libraryID 200
query_item_data = '''
    SELECT id.itemID, fc.fieldName, idv.value
    FROM itemData id
    JOIN fieldsCombined fc ON id.fieldID = fc.fieldID
    JOIN itemDataValues idv ON id.valueID = idv.valueID
    JOIN items i ON id.itemID = i.itemID
    WHERE i.libraryID = ?;
'''
df_item_data = pd.read_sql_query(query_item_data, conn, params=(library_id,))

# Pivot the item data DataFrame
df_item_data_pivot = df_item_data.pivot(index='itemID', columns='fieldName', values='value').reset_index()

# Close the connection to the database
conn.close()

# Merge the DataFrames
df_zotero = pd.merge(df_authors, df_item_data_pivot, on='itemID', how='left')
df_zotero = pd.merge(df_zotero, df_paths, left_on='itemID', right_on='parentItemID', how='left').drop('parentItemID', axis=1)

# Display the first few rows of the final DataFrame
df_zotero.head()

,itemID,libraryID,author_last_name,author_first_name,DOI,ISBN,ISSN,abstractNote,accessDate,bookTitle,...,proceedingsTitle,publicationTitle,publisher,shortTitle,thesisType,title,university,url,volume,path
0,1021983,200,Tomes,Robert R.,NaN,NaN,0095-327X,"Conflicting time and space perceptions, seated...",2024-01-12 00:37:51,NaN,...,NaN,Armed Forces & Society,NaN,NaN,NaN,Space-Time Orientations and Contemporary Polit...,NaN,https://www.jstor.org/stable/48616598,31,storage:Cunningham and Tomes - 2004 - Space-Ti...
1,1021983,200,Cunningham,Kevin,NaN,NaN,0095-327X,"Conflicting time and space perceptions, seated...",2024-01-12 00:37:51,NaN,...,NaN,Armed Forces & Society,NaN,NaN,NaN,Space-Time Orientations and Contemporary Polit...,NaN,https://www.jstor.org/stable/48616598,31,storage:Cunningham and Tomes - 2004 - Space-Ti...
2,1021885,200,Solomon,Ty,10.1111/isqu.12091,NaN,0020-8833,Critically inclined International Relations (I...,2024-01-12 00:09:20,NaN,...,NaN,International Studies Quarterly,NaN,NaN,NaN,Time and Subjectivity in World Politics,NaN,https://doi.org/10.1111/isqu.12091,58,storage:Solomon - 2014 - Time and Subjectivity...
3,1021908,200,Amoureux,Jack L.,NaN,NaN,NaN,NaN,2024-01-12 00:13:04,NaN,...,NaN,International Theory,NaN,Is Faster Better?,NaN,Is Faster Better? Political and Ethical Framin...,NaN,https://www.cambridge.org/core/journals/intern...,12,storage:Amoureux - 2020 - Is faster better Pol...
4,1021971,200,Neumann,Iver B.,10.1177/0305829818771339,NaN,"0305-8298, 1477-9021","Drawing on identity and prototype theory, the ...",2024-01-12 00:31:09,NaN,...,NaN,Millennium: Journal of International Studies,NaN,Halting Time,NaN,Halting Time: Monuments to Alterity,NaN,http://journals.sagepub.com/doi/10.1177/030582...,46,storage:Neumann - 2018 - Halting Time Monument...


In [19]:
import pandas as pd
import os
from datetime import datetime

# Get the current date in YYMMDD format
date_suffix = datetime.now().strftime('%y%m%d')

# Define the file path with the date suffix
output_file_path = f'Temporality_zotero_library_{date_suffix}.jsonl'

# Write the DataFrame to a JSONL file without ASCII output commenting
df_zotero.to_json(output_file_path, orient='records', lines=True, force_ascii=False, date_format='iso')

# Verify that the file has been created
print(f'File "{output_file_path}" has been created.')

File "Temporality_zotero_library_240113.jsonl" has been created.
